In [21]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict

In [22]:
def getCDF(data):
    xdata = np.sort(data)
    ydata = [i/len(xdata) for i in range(len(xdata))]
    return xdata, ydata

In [24]:
DNS_moderate_lookups = {
    'nest_therm': '.nest.com',
    'nest_cam': '.dropcam.com',
    'smartthings': '.smartthings.com',
    'pixstar': '.pix-star.com',
    'echo': 'device-metrics-us.amazon.com',
    'sharx': 'sharxsecurity.com',
    'august': '.august.com',
    'hue': '.meethue.com',
    #'apple': 'time-ios.apple.com',
    'apple': 'gs-loc.ls-apple.com',
    'ps': '.playstation.net',
    'xbox': 'xboxlive.com',
    'nintendo': 'nintendo.net',
    }

In [25]:
# LOAD PIMA EXTRACT
df = pd.read_csv("pima-extract.csv", sep="|").fillna('')

In [26]:
def reduce_domain(dom):
    # return 2nd level domain
    return ".".join(dom.split('.')[-3:])

In [27]:
df['domain'] = df['dns.qry.name'].apply(lambda x: reduce_domain(x))

In [28]:
# REMOVE entries with ipv6
df2 = df[df['ip.src']!='']

In [29]:
df2.head()

,frame.time_relative,eth.src,eth.dst,ip.src,ip.dst,dns.qry.name,domain
0,0.000000,00:1f:a0:11:b5:3a,40:a8:f0:34:48:5c,24.9.76.250,69.252.212.78,android.clients.google.com,clients.google.com
1,0.000001,00:1c:73:69:93:01,e0:ac:f1:39:39:53,69.241.95.82,69.31.113.189,e3645.b.akamaiedge.net,b.akamaiedge.net
2,0.000005,00:1f:a0:11:b5:3a,40:a8:f0:34:19:30,24.9.76.250,69.252.212.88,android.clients.google.com,clients.google.com
3,0.000009,00:1f:a0:11:b5:3a,40:a8:f0:34:1f:e0,24.9.76.250,69.252.212.84,android.clients.google.com,clients.google.com
4,0.000010,00:1f:a0:11:b5:3a,40:a8:f0:33:fa:68,73.153.66.183,69.252.212.76,www.polls.newsvine.com,polls.newsvine.com


# SERVER ANALYSIS
- why 11k resolvers?

In [30]:
len( df2['ip.dst'].unique() )

11766

In [31]:
# By number of queries
l = df2.groupby('ip.dst')['domain'].count()
l.sort(ascending=False)
print len(l)
l

11766


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting
  app.launch_new_instance()


ip.dst
69.252.212.70      148687
69.252.212.78      148680
69.252.212.76      148669
69.252.212.72      148666
69.252.212.74      148659
69.252.212.66      148652
69.252.212.68      148650
69.252.212.88      148649
69.252.212.80      148643
69.252.212.82      148640
69.252.212.84      148639
69.252.212.86      148638
68.87.85.132         6773
205.251.195.166      6247
205.251.195.252      4032
192.48.79.30         3179
192.54.112.30        3124
69.252.205.234       3000
63.243.243.14        2891
192.35.51.30         2617
216.239.34.10        2310
192.12.94.30         2224
199.7.83.42          2117
84.53.139.129        2089
205.251.194.154      1949
205.251.194.48       1719
204.79.195.48        1701
96.17.70.171         1611
192.42.93.30         1543
63.243.243.13        1479
                    ...  
205.251.198.19          1
205.251.198.191         1
205.251.198.195         1
205.251.198.21          1
205.251.198.212         1
205.251.198.140         1
205.251.198.138         1
205.2

In [32]:
# by source-dst combos
l = df2.groupby(['ip.dst', 'ip.src'])['domain'].count()
l.sort(ascending=False)
l

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting
  app.launch_new_instance()


ip.dst         ip.src        
69.252.212.78  73.14.106.0       13202
69.252.212.80  73.14.106.0       13100
69.252.212.74  73.14.106.0       13092
69.252.212.66  73.14.106.0       13082
69.252.212.70  73.14.106.0       13065
69.252.212.82  73.14.106.0       13060
69.252.212.72  73.14.106.0       13059
69.252.212.88  73.14.106.0       13057
69.252.212.76  73.14.106.0       13044
69.252.212.86  73.14.106.0       13030
69.252.212.84  73.14.106.0       12969
69.252.212.68  73.14.106.0       12955
69.252.212.70  71.228.150.166     8277
69.252.212.66  71.228.150.166     8261
69.252.212.72  71.228.150.166     8224
69.252.212.74  71.228.150.166     8200
69.252.212.68  71.228.150.166     8197
69.252.212.88  71.228.150.166     8197
69.252.212.76  71.228.150.166     8179
69.252.212.86  71.228.150.166     8165
69.252.212.84  71.228.150.166     8087
69.252.212.80  71.228.150.166     8054
69.252.212.82  71.228.150.166     8053
69.252.212.78  71.228.150.166     8011
69.252.212.80  67.162.143.192     

In [33]:
m = l.reset_index().groupby('ip.dst')['domain'].count()
m.sort(ascending=False)
print len(m)
m

11766


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting
  from ipykernel import kernelapp as app


ip.dst
69.252.212.66      65721
69.252.212.70      65654
69.252.212.72      65633
69.252.212.82      65604
69.252.212.80      65594
69.252.212.74      65589
69.252.212.88      65563
69.252.212.68      65557
69.252.212.76      65521
69.252.212.78      65484
69.252.212.84      65356
69.252.212.86      65234
209.234.234.42         6
209.67.208.4           6
209.66.70.22           6
173.227.191.13         6
209.82.241.34          6
209.249.181.21         6
209.244.4.149          6
209.244.0.1            6
205.251.195.57         6
209.211.200.8          6
209.175.46.21          6
209.170.216.2          6
210.116.114.212        6
209.17.114.135         6
209.144.103.30         6
209.141.51.61          6
210.103.240.10         6
172.17.22.11           6
                   ...  
207.45.178.157         1
207.44.96.129          1
207.44.208.37          1
207.44.188.90          1
207.210.151.9          1
207.210.202.217        1
207.210.70.162         1
207.226.161.2          1
207.230.136.194   

### Unique DNS servers analysis
- Over 11k servers for 2500k homes, 500k were ipv6
- 12 main servers for 65k homes each


# Identifying IoT devices using DNS data


## Hypothesis 1: There will not be many homes with IoT related DNS queries
- most popular domains across homes in dataset will be non-IoT related and can be filtered out using a popularity threshold 
- Count second-level domains across homes - IoTs should be in the filtered tail

In [34]:
gp = df2.groupby(['ip.src', 'domain'])
df_home_dom = gp['dns.qry.name'].count().reset_index()

In [35]:
print len(df_home_dom['ip.src'].unique())
print len(df_home_dom['domain'].unique())

260840
72964


In [36]:
domain_home_count = df_home_dom.groupby('domain')['ip.src'].count()
domain_home_count.sort(ascending=False)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting
  from ipykernel import kernelapp as app


In [57]:
domain_home_count[domain_home_count>2000]

domain
com.akadns.net                  42301
dscc.akamaiedge.net             21164
www.google.com                  14996
g.aaplimg.com                   14748
icloud.apple-dns.net            14378
www.apple.com                   13831
e9.akamaiedge.net               13538
r53.xcal.tv                     13479
l.google.com                    12457
push.apple.com                  11833
apple.com                       10187
c10r.facebook.com                8580
elb.amazonaws.com                8098
clients.google.com               7904
graph.facebook.com               7596
a.root-servers.net               7592
g.akamaiedge.net                 7215
time-ios.apple.com               6642
www.belkin.com                   6561
co.comcast.net                   6122
guzzoni.apple.com                6106
itunes.apple.com                 5906
smoot.apple.com                  5431
fna.fbcdn.net                    5398
www.googleapis.com               5376
a.akamaiedge.net                 5168
ccp.x

In [50]:
for indx, val in DNS_moderate_lookups.items():
    if val[0] == '.':
        val2 = val[1:]
    else:
        val2 = val
    if ( val2 in domain_home_count ):
        print "device:", indx, "domain:", val2, "count:", domain_home_count.ix[val2]

meethue.com
gs-loc.ls-apple.com
device-metrics-us.amazon.com
device: echo domain: device-metrics-us.amazon.com count: 2041
sharxsecurity.com
nintendo.net
dropcam.com
august.com
playstation.net
smartthings.com
pix-star.com
xboxlive.com
nest.com


In [54]:
for k in domain_home_count.keys():
    for indx, val in DNS_moderate_lookups.items():
        if val in k:
            print domain_home_count[k],  indx, val, k

2041 echo device-metrics-us.amazon.com device-metrics-us.amazon.com
644 ps .playstation.net dl.playstation.net
380 ps .playstation.net community.playstation.net
301 ps .playstation.net km.playstation.net
296 nest_therm .nest.com devices.nest.com
273 xbox xboxlive.com auth.xboxlive.com
196 ps .playstation.net update.playstation.net
176 xbox xboxlive.com licensing.xboxlive.com
168 nintendo nintendo.net app.nintendo.net
163 xbox xboxlive.com update-cdn.xboxlive.com
158 xbox xboxlive.com www.xboxlive.com
149 nest_therm .nest.com home.nest.com
132 nintendo nintendo.net cdn.nintendo.net
130 nintendo nintendo.net shop.nintendo.net
120 ps .playstation.net communication.playstation.net
115 ps .playstation.net ad.playstation.net
114 nintendo nintendo.net olv.nintendo.net
113 xbox xboxlive.com eplists.xboxlive.com
102 ps .playstation.net ac.playstation.net
97 xbox xboxlive.com assets1.xboxlive.com
93 xbox xboxlive.com titlestorage.xboxlive.com
93 xbox xboxlive.com settings.xboxlive.com
90 xbox xb

### Threshold and remove
- thresh < 10000

In [42]:
domain_home_count[ domain_home_count < 10000 ]

domain
c10r.facebook.com                 8580
elb.amazonaws.com                 8098
clients.google.com                7904
graph.facebook.com                7596
a.root-servers.net                7592
g.akamaiedge.net                  7215
time-ios.apple.com                6642
www.belkin.com                    6561
co.comcast.net                    6122
guzzoni.apple.com                 6106
itunes.apple.com                  5906
smoot.apple.com                   5431
fna.fbcdn.net                     5398
www.googleapis.com                5376
a.akamaiedge.net                  5168
ccp.xcal.tv                       4470
g.doubleclick.net                 4210
edge-mqtt.facebook.com            4191
com.edgekey.net                   3750
ls.apple.com                      3738
api-global.netflix.com            3736
data.microsoft.com                3456
imap.gmail.com                    3440
ichnaea.netflix.com               3250
l.doubleclick.net                 3222
clients4.google.co